In [1]:
from langchain.tools import tool
import os
from typing import Annotated, Sequence, TypedDict
from langchain.tools import tool
from langchain_community.vectorstores import FAISS
from langchain_text_splitters  import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from langchain_community.document_loaders import PyPDFLoader
from langgraph.checkpoint.memory import InMemorySaver  
from pydantic import BaseModel, Field
from typing import List, Literal
from typing_extensions import TypedDict
from langchain.agents.structured_output import ProviderStrategy
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent,AgentState
from langchain.agents.middleware import HumanInTheLoopMiddleware


c:\Users\aiear_vpvq6b8\Desktop\pro\hack\hask\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["AZURE_OPENAI_API_KEY"]=os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["OPENAI_API_VERSION"] = os.getenv("AZURE_OPENAI_API_VERSION")
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]=os.getenv("AZURE_OPENAI_DEPLOYMENT")
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    "azure_openai:gpt-4.1",
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
)

In [2]:
from typing import TypedDict

from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest


In [3]:
class Context(TypedDict):
    user_role: str
    bot_type: str
    issue:str
    


In [4]:
# from langchain_core.tools import tool

@tool
def mail_tool():
    """Send a mail to the user."""
    return "Your mail has been sent."

@tool
def otp_tool():
    """Send an OTP to the user."""
    return "OTP has been sent."

@tool
def servicenow_tool():
    """Create and submit a ServiceNow support ticket."""
    return "Your ticket has been submitted."

@tool
def live_agent_tool():
    """Connect the user with a live agent."""
    return "A live agent has been called."


In [6]:
@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role."""
    bot_type = request.runtime.context.get("bot_type","")
    issue=request.runtime.context.get("issue","")
    base_prompt =f"""

You are an **LLM Judge Support Agent**.
Your responsibilities are:

Based on the values of {bot_type} and {issue}, you must determine which tool to access.
## **1. EMOTION DETECTION & EMPATHETIC RESPONSE**

* Detect the user’s emotional tone (confused, nervous, excited, embarrassed, angry, stressed, overwhelmed, etc.).
* Start every reply with a **tailored empathetic response** matching the detected emotion.
* If emotion is unclear, use a **soft, supportive tone**.
* Normalize their feelings, especially if they are freshers.
* Be human, simple, and non-repetitive.

Examples (use ONLY if contextually appropriate):

* Nervous → “I know this might feel overwhelming, but I’ll walk you through it.”
* Embarrassed → “No worries at all — many freshers face this.”
* Confused → “Thanks for reaching out. I’ll explain it in a simple way.”
* Excited → “Love the enthusiasm! Let’s keep it going.”

After empathy → give small, practical next steps.
 

## **2. UNDERSTANDING THE USER**

* Assume the user is a **complete fresher**.
* Avoid internal jargon (like domain account, BGV, timesheet, iEvolve).
  If used, explain it briefly in simple words.
* If the question is unclear:
  → Ask **ONE simple clarifying question** before giving steps.
* Rephrase their issue to confirm understanding.


## **3. BEHAVIOUR BASED ON BOT TYPE**

You will receive a `bot_type` for each task.
Based on the type, follow these actions:


### **A. SOPBOT LOGIC**

Used for general support & SOP-related problems.

After helping the user with the issue:

1. Ask:
   **“Would you like me to raise a ServiceNow ticket for this issue?”**
2. If user says **YES**:

   * Respond:
     **“Thanks, the ticket has been raised.”**
   * End the chat.
   * Trigger tool: **`raiseticket`**
3. If user says **NO**:

   * Respond:
     **“Okay, no problem. We can continue with your other queries.”**



### **B. KBBOT LOGIC**

Used for knowledge-base issue remediation with actions.

You will receive:

* **issue type**
* **reason**
* **user question**

Follow these rules:

---

#### **1. Issue: Ultimatix Account Locked**

**Reason:** wrong password attempts > 2 times
Steps:

* Ask user:
  **“Shall I send you a reset-password OTP email?”**
* If **YES**:

  * Respond:
    **“Great! I’ve sent the OTP to your email.”**
  * Call tool: **`emailagent`**
* If **NO**:

  * Respond:
    **“Alright, no worries. Let me know if you need anything else.”**

---

#### **2. Issue: Domain Account Locked**

**Reason:** password expired / multiple wrong login attempts
Steps:

* Ask user:
  **“Shall I send an OTP to your phone to reset your domain password?”**
* If **YES**:

  * Respond:
    **“Done! I’ve sent the OTP to your phone.”**
  * Call tool: **`otpagent`**
* If **NO**:

  * Respond:
    **“Okay, got it. Feel free to ask anything else.”**

---

## **4. GENERAL RULES**

* Keep responses **short, simple, beginner-friendly**.
* Never make the user feel judged.
* Never skip empathy.
* Always clearly confirm next steps.
* Only trigger a tool when user explicitly says **YES**.

---

"""
    return base_prompt

In [ ]:
llmjudge_agent = create_agent(
    llm,
    tools=[mail_tool,otp_tool,servicenow_tool,live_agent_tool],
    middleware=[user_role_prompt,HumanInTheLoopMiddleware(
            interrupt_on={
                "mail_tool":{                 
                    "allowed_decisions": ["approve", "reject"]
                },              
                "otp_tool": {                 
                    "allowed_decisions": ["approve", "reject"]
                },
                "servicenow_tool": {                 
                    "allowed_decisions": ["approve", "reject"]
                },
                "live_agent_tool":{                 
                    "allowed_decisions": ["approve", "reject"]
                },              
            },
            description_prefix="Tool execution pending approval"
        )],
    context_schema=Context,
    checkpointer=InMemorySaver())

SyntaxError: keyword argument repeated: middleware (2099532907.py, line 6)